In [ ]:
import torch
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.manual_seed(777)
if device == "cuda":
  torch.cuda.manual_seed_all(777)

model = torch.load('resnet101.pt', device = device)

In [ ]:
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

test_path = './test'

In [ ]:
resize_trans = transforms.Compose([
	transforms.Resize((256,256)),
 transforms.ToTensor()
])

resize_test = torchvision.datasets.ImageFolder(root = test_path, transform = resize_trans)

transform_test = transforms.Compose([
	transforms.Resize((256,256)),
 	transforms.CenterCrop(224),
	transforms.ToTensor(),
  transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

testset = torchvision.datasets.ImageFolder(root = test_path, transform=transform_test)

testloader = DataLoader(testset, batch_size=4, shuffle=False)

In [ ]:
def get_representation(model, iterator):
  
  model.eval()
  
  outputs = []
  labels = []
  
  with torch.no_grad():
    for (x,y) in iterator:
      
      x = x.to(device)
      
      y_pred = model(x)
      
      outputs.append(y_pred.cpu())
      labels.append(y)
      
  outputs = torch.cat(outputs, dim = 0)
  labels = torch.cat(labels, dim = 0)
  
  return outputs, labels

In [ ]:
testoutputs, testlabels = get_representation(model, testloader)

In [ ]:
import sklearn
from sklearn import manifold
import matplotlib.pyplot as plt

In [ ]:
def plot_representation(data, labels, classes=None,n_Images=None):
  if n_Images is not None:
    data = data[:n_Images]
    labels = labels[:n_Images]
    
  fig = plt.figure(figsize = (10,10))
  ax = fig.add_subplot(111)
  scatter = ax.scatter(data[:,0],data[:,1])

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler

output_scaled = StandardScaler().fit_transform(testoutputs)
lda = LinearDiscriminantAnalysis(n_component = 2)
lda.fit(output_scaled, testlabels)

output_lda = lda.transform(output_scaled)
plot_representation(output_lda, testlabels)

In [ ]:
tsne = manifold.TSNE(n_components=2, learning_rate = 200)
output_tsne = tsne.fit_tranform(output_scaled)
plot_representation(output_tsne, testlabels)

In [ ]:
from typing import Text
from sklearn.cluster import MeanShift, estimate_bandwidth
from itertools import cycle

bandwidth = estimate_bandwidth(output_lda, quantile = .1)
ms = MeanShift(bandwidth = bandwidth + .3)
ms.fit(output_lda)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
clusters = labels.max()+1

color = cycle('rgbcmyk')
for i, col in zip(range(clusters),color):
  my_member = labels == i
  cluster_center = cluster_centers[i]
  plt.plot(output_lda[my_member,0], output_lda[my_member,1],col + '.')
  plt.plot(cluster_center[0], cluster_center[1], 'o', 
           markerfacecolor = col, markeredgecolor = 'k', markersize = 12)
  print('Estimated clusters : %d' % clusters)
  
  plt.show()

In [ ]:
bandwidth = estimate_bandwidth(output_tsne, quantile = .1)
ms = MeanShift(bandwidth = bandwidth + .3)
ms.fit(output_tsne)
labels = ms.labels_
cluster_centers = ms.cluster_centers_
clusters = labels.max()+1

color = cycle('rgbcmyk')
for i, col in zip(range(clusters),color):
  my_member = labels == i
  cluster_center = cluster_centers[i]
  plt.plot(output_tsne[my_member,0], output_tsne[my_member,1],col + '.')
  plt.plot(cluster_center[0], cluster_center[1], 'o', 
           markerfacecolor = col, markeredgecolor = 'k', markersize = 12)
  print('Estimated clusters : %d' % clusters)
  
  plt.show()